# TrackML Challenge Kernel for Kaggle Course 

In [ ]:
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))

This kernel outlines the process taken towards the Kaggle Course offered by AI Academy at Aalto University.

**Due to the nature of the TrackML Challenge, this notebook will deviate from the requirements to clearly present the problem and associated soltuions that were tested. Please read the rest of the document and evaluate it accordingly.**

I will also be extensively using content from public kernels that explain the data and problem much better than I ever can hope to.

## Problem Overview
The TrackML Challenge was put forth by CERN to find solutions to the well known particle path reconstruction problem. The problem will be explained in stages. 

Firstly an into to CERN, they have large particle colliders that speed up elementary particles to high speeds and then collide them and find inferences from the data generated from the collision. 

Each collision results in a myriad of particles that are fleeting in nature and then decompose into even more particles. The detectors in CERN are **silicon slabs** that are able to detect small changes that correspond to when a particle hits the detector. The detectors are arranged in a manner as shown in the figure below. <br>
<img src="https://i.imgur.com/vse3EMh.png" width=500>
<br>
This gives us an view of how the detectors are arranged in 3 dimensions, which will come in handy when doing visualizations.A very simple analogy to understand the problem:<br>


**Consider that we shoot a bullet through multiple vertical cardboard cutouts that are separated horizonatally. Now after the bullet has passed through them we are left with the holes in the cardboard, now our job is to reconstruct the path of the bullet given the holes detected in the cardboard**. 

The following image may provide an intuition behind the problem, where the red dots are the holes left behind and the arc is the possible part of the bullet.
<br>
<img src ="https://i.imgur.com/CJynDG6.png" >

## Data
Upon getting an understanding of the problem, it may be easy to ask, **what is so hard about this?**. A valid question, this brings us to the complexity of the problem and the difficulty in directly throwing any well known ML algorithm without thinking. This is due to the enormous amount of data that is generated and given to us.<br>
Consider each collision as an **event**. Each **event** creates roughly **10,000** particles. Each particle passes through an average of 10 silicon detectors. On average each **event** has **100,000** data points. The whole dataset has **8,850** events leading to a grand total of **70GB** of training data. 

* __Hits__ are given as x, y, z positions and the detector groups they interacted with: volume, layer, module in increasing granularity (volume big, module small). 
* For each hit, we're also also given the __cells__ position and charge that did the detecting. These are the smallest unit of detector group, even finer than module. 
* For each event we're also given __particle__ information x, y, z momentums, and charge, and number of detectors hit from that particle.  
* Finally, we're given __truth__, telling us for certain which hits belong to which particles. Maybe this is kind of like our "y", and the other features are kind of like our "X".

This is all explained in more detail below, later.

First let's see some visualizations

In [ ]:
from trackml.dataset import load_event
import os
import seaborn as sns
import matplotlib.pyplot as plt
import scipy.stats as stats
import numpy as np
import pandas as pd
from IPython.display import Image,display
from trackml.dataset import load_dataset
from trackml.score import score_event

In [ ]:
import matplotlib as mpl
from mpl_toolkits.mplot3d import Axes3D

mpl.rcParams['figure.dpi'] = 300

## Load one event to create visualizations

In [ ]:
event_prefix = 'event000001000'
path_to_train = "../input/trackml-particle-identification/train_1"
path_to_test = "../input/trackml-particle-identification/test/"
hits, cells, particles, truth = load_event(os.path.join('../input/trackml-particle-identification/train_1/', event_prefix))
hits.describe()

__Credits to : @Joshua Bonatt <br>
__hits__  
The hits file contains the following values for each hit/entry:  
* __hit_id__: numerical identifier of the hit inside the event.
* __x, y, z__: measured x, y, z position (in millimeter) of the hit in global coordinates.
* __volume_id__: numerical identifier of the detector group.
* __layer_id__: numerical identifier of the detector layer inside the group.
* __module_id__: numerical identifier of the detector module inside the layer.  

The volume/layer/module id could in principle be deduced from x, y, z. They are given here to simplify detector-specific data handling.

__cells__  
The cells file contains the constituent active detector cells that comprise each hit. The cells can be used to refine the hit to track association. A cell is the smallest granularity inside each detector module, much like a pixel on a screen, except that depending on the volume_id a cell can be a square or a long rectangle. It is identified by two channel identifiers that are unique within each detector module and encode the position, much like column/row numbers of a matrix. A cell can provide signal information that the detector module has recorded in addition to the position. Depending on the detector type only one of the channel identifiers is valid, e.g. for the strip detectors, and the value might have different resolution.  
* __hit_id__: numerical identifier of the hit as defined in the hits file.
* __ch0, ch1__: channel identifier/coordinates unique within one module.
* __value__: signal value information, e.g. how much charge a particle has deposited.

__particles__  
The particles files contains the following values for each particle/entry:  
* __particle_id__: numerical identifier of the particle inside the event.
* __vx, vy, vz__: initial position or vertex (in millimeters) in global coordinates.
* __px, py, pz__: initial momentum (in GeV/c) along each global axis.
* __q__: particle charge (as multiple of the absolute electron charge).
* __nhits__: number of hits generated by this particle.

__truth__  
The truth file contains the mapping between hits and generating particles and the true particle state at each measured hit. Each entry maps one hit to one particle.  
* __hit_id__: numerical identifier of the hit as defined in the hits file.
* __particle_id__: numerical identifier of the generating particle as defined in the particles file. A value of 0 means that the hit did not originate from a reconstructible particle, but e.g. from detector noise.
* __tx, ty, tz__ true intersection point in global coordinates (in millimeters) between the particle trajectory and the sensitive surface.
* __tpx, tpy, tpz__ true particle momentum (in GeV/c) in the global coordinate system at the intersection point. The corresponding vector is tangent to the particle trajectory at the intersection point.
* __weight__ per-hit weight used for the scoring metric; total sum of weights within one event equals to one.



## Hits

In [ ]:
hits.head()

In [ ]:
hits.tail()

In [ ]:
g = sns.jointplot(hits.x, hits.y,  s=1, height=12)
g.ax_joint.cla()
plt.sca(g.ax_joint)

volumes = hits.volume_id.unique()
for volume in volumes:
    v = hits[hits.volume_id == volume]
    plt.scatter(v.x, v.y, s=3, label='volume {}'.format(volume))

plt.xlabel('X (mm)')
plt.ylabel('Y (mm)')
plt.legend()
plt.show()

**The solid green core in the middle is more concentric tigtly packed detectors. Let's zoom into them and see them.**

In [ ]:
g = sns.jointplot(hits.x, hits.y,  s=1, height=12)
g.ax_joint.cla()
plt.sca(g.ax_joint)

volumes = hits.volume_id.unique()
for volume in volumes:
    v = hits[hits.volume_id == volume]
    plt.scatter(v.x, v.y, s=3, label='volume {}'.format(volume))

plt.xlim([-200,200])
plt.ylim([-200,200])
plt.xlabel('X (mm)')
plt.ylabel('Y (mm)')
plt.legend()
plt.show()

That scattering we see (the blue and green dots) in between the orange rings (detectors) are just events detected in the vertical caps. Upon Removing them we see only the detections on the X and Y planes. We set a value of **|z|<=300**, these are the locations of the first set of vertical detectors, hence we remove all blue and green events.

Upon even closer observation, each orange slab is actually a detector that has been saturated with hits to become visible

In [ ]:
g = sns.jointplot(hits.x, hits.y,  s=1, height=12)
g.ax_joint.cla()
plt.sca(g.ax_joint)

volumes = hits.volume_id.unique()
for volume in volumes:
    v = hits[hits.volume_id == volume]
    plt.scatter(v[v.z.abs()<300].x, v[v.z.abs()<300].y, s=3, label='volume {}'.format(volume))

plt.xlim([-200,200])
plt.ylim([-200,200])
plt.xlabel('X (mm)')
plt.ylabel('Y (mm)')
plt.legend()
plt.show()

## A view along Y and Z axes

Now we can see the orange detectors from before as the concentric rings from above. We can now also see the position of the vertical detectors along the Z axis in blue and green near the origin. 

In [ ]:
volumes = hits.volume_id.unique()
plt.figure(figsize=(24,8))

for volume in volumes:
    v = hits[hits.volume_id == volume]
    plt.scatter(v.z, v.y, s=1, label='volume {}'.format(volume))

plt.xlabel('Z (mm)')
plt.ylabel('Y (mm)')
plt.ylim([-200,200])
plt.legend()
plt.show()


## Undertanding Concentric arrangement

If we take apart the concentric orange rings and then plot them according to the volume id that denotes the location of the detectors. 

__Volume 7__ is to the __left__, __Volume 8__ is the __middle__ (concentric in X axis) and __Volume 9__ in the __right__. Similarly the next 3 volume ids also follow the same order.

We see that __Volume 8,13,17__ are the ones that have concentric rings of detectors along the X axis

In [ ]:
g = sns.FacetGrid(hits, col="volume_id",aspect=0.5,sharey=True,sharex=True)
g=(g.map(plt.scatter,"x", "y",s=0.01))
g.set_axis_labels('')
for i in range(len(hits.volume_id.unique())):
    g.facet_axis(0,i).axis('off')
plt.tight_layout()
plt.show()

### A 3D plot of volume ids of the whole detector

In [ ]:
fig = plt.figure(figsize=(12, 12))
ax = fig.add_subplot(111, projection='3d')
sample = hits.sample(10000)
for volume in volumes:
    v = sample[sample.volume_id == volume]
    ax.scatter(v.z, v.x, v.y, s=1, label='volume {}'.format(volume), alpha=0.5)
ax.set_title('Hit Locations')
ax.set_xlabel('Z (millimeters)')
ax.set_ylabel('X (millimeters)')
ax.set_zlabel('Y (millimeters)')
#ax.view_init(elev=10,azim=10)
ax.legend()

ax.scatter(3000,3000,3000, s=0)
ax.scatter(-3000,-3000,-3000, s=0)
plt.show()

### A 3D plot for Layer IDs 

Layer IDs are different from volume and as seen in the plot, the increasing Layer ID indicates how close to the centre they were.

In [ ]:
fig = plt.figure(figsize=(12, 12))
ax = fig.add_subplot(111, projection='3d')
layers = hits.layer_id.unique()

for layer in layers:
    v = sample[sample.layer_id == layer]
    ax.scatter(v.z, v.x, v.y, s=1, label='layer {}'.format(layer), alpha=0.5)
ax.set_title('Hit Locations')
ax.set_xlabel('Z (millimeters)')
ax.set_ylabel('X (millimeters)')
ax.set_zlabel('Y (millimeters)')
ax.scatter(3000,3000,3000, s=0)
ax.scatter(-3000,-3000,-3000, s=0)
ax.legend()
#ax.view_init(elev=10,azim=10)
plt.show()

# <a name="particles">Particles</a>
The particles files contains the following values for each particle/entry:  
* __particle_id__: numerical identifier of the particle inside the event.
* __vx, vy, vz__: initial position or vertex (in millimeters) in global coordinates.
* __px, py, pz__: initial momentum (in GeV/c) along each global axis.
* __q__: particle charge (as multiple of the absolute electron charge).
* __nhits__: number of hits generated by this particle.

In [ ]:
particles.describe()

# <a name="truth">Truth</a>
The truth file contains the mapping between hits and generating particles and the true particle state at each measured hit. Each entry maps one hit to one particle.  
* __hit_id__: numerical identifier of the hit as defined in the hits file.
* __particle_id__: numerical identifier of the generating particle as defined in the particles file. A value of 0 means that the hit did not originate from a reconstructible particle, but e.g. from detector noise.
* __tx, ty, tz__ true intersection point in global coordinates (in millimeters) between the particle trajectory and the sensitive surface.
* __tpx, tpy, tpz__ true particle momentum (in GeV/c) in the global coordinate system at the intersection point. The corresponding vector is tangent to the particle trajectory at the intersection point.
* __weight__ per-hit weight used for the scoring metric; total sum of weights within one event equals to one.

In [ ]:
truth.head()

# True Particle Trajectories

This plot shows what we wish to achieve, the true paths of the particles given the hits and particels information. This is only a plot of 100th particle track

In [ ]:
# Get every 100th particle

tracks = truth.particle_id.unique()[1::100]

plt.figure(figsize=(15,15))
ax = plt.axes(projection='3d')
for track in tracks:
    t = truth[truth.particle_id == track]
    ax.plot3D(t.tz, t.tx, t.ty)
ax.set_xlabel('z (mm)')
ax.set_ylabel('x (mm)')
ax.set_zlabel('y (mm)')
# These two added to widen the 3D space
ax.scatter(3000,3000,3000, s=0)
ax.scatter(-3000,-3000,-3000, s=0)
plt.show()

## Some Physics (Unfortunately) for Feature Engineering and Selection

By utilizing some particle physics we can come up with ways to convert the hit locations into coordinates that may be easier to group. This rests on the idea that a charged particle moving in a magnetic field will assume a helical orbit as seen in the image below. 
<br>
<img src= "https://i.imgur.com/2TEpdjr.png">
<br>

As the slide from the High Energy Physis (HEP) domain suggests by converting our cartesian coordinates to other coordinates we can achieve a better grouping of the __hits__ that are detected.

The following involve some partice physics calculations that show how we arive at the transformation that is the easiest to implement. 

## Coordinate Transformations
**Credits: @Mark JD Hamilton**<br>
**Can Be Skipped** <br>
Some notations to keep in common 
We fix the following notation:
1. Particle rest mass $m_0$ and electric charge $q$.
2. $m=\gamma m_0$ with relativistic factor $\gamma$ (considered constant).
3. Coordinate system $(x,y,z)$.
4. Magnetic field $\vec{B}=Be_z$.
5. Initial coordinates $x_0=y_0=z_0=0$ (for simplicity, see comment below).
6. Initial velocities $v_{x0}, v_{y0}, v_{z0}$ and initial momenta $p_{x0}, p_{y0}, p_{z0}$ with $p_{i0}=m v_{i0}$ for $i=x,y,z$.
7. Longitudinal momentum $p_\parallel = p_{z0}$, transversal momentum 
\begin{equation*}
p_\perp = \sqrt{p_{x0}^2+p_{y0}^2},
\end{equation*}
and total momentum 
\begin{equation*}
p=\sqrt{p_\parallel^2+p_\perp^2}.
\end{equation*}

We define:
\begin{align*}
\omega &= \frac{qB}{m}\\
R&= \frac{p_\perp}{qB}
\end{align*}
and an angle $\phi_0$ by
\begin{align*}
\cos\phi_0&=-\frac{p_{y0}}{p_\perp}\\
\sin\phi_0&=\frac{p_{x0}}{p_\perp}.
\end{align*}
The system of differential equations
\begin{align*}
m\ddot{x} &= qB\dot{y}\\
m\ddot{y} &= -qB\dot{x}\\
m\ddot{z} &= 0
\end{align*}
has the helix solution
\begin{align*}
x(t) &= R(\cos(\phi_0-\omega t)-\cos(\phi_0))\\
y(t) &= R(\sin(\phi_0-\omega t)-\sin(\phi_0))\\
z(t) &= R\frac{p_\parallel}{p_\perp}\omega t.
\end{align*}
If the initial coordinates are non-zero, we just add $x_0$, $y_0$ and $z_0$ to these expressions.

Projected onto the $x$-$y$-plane, the helix forms a circle of radius $|R|$ with center at the point 
\begin{equation*}
-R(\cos\phi_0,\sin\phi_0).
\end{equation*}
The motion in $z$-direction is of constant velocity.

Using
\begin{align*}
\cos^2(\alpha)+\sin^2(\alpha)&=1\\
\cos(\alpha)\cos(\beta)+\sin(\alpha)\sin(\beta)&=\cos(\alpha-\beta),
\end{align*}
we have
\begin{align*}
x^2+y^2 &= 2R^2(1-\cos(\omega t))\\
&\approx R^2\left(\omega^2t^2-\frac{1}{12}\omega^4t^4  \right),
\end{align*}
where the approximation in the second line follows from Taylor's formula and is valid for small $\omega t$.

We define a new coordinate
\begin{equation*}
z_2=\frac{z}{\sqrt{x^2+y^2}}=\pm\frac{p_\parallel}{p_\perp}\frac{\omega t}{\sqrt{2(1-\cos(\omega t))}},
\end{equation*}
which is independent of $R$ and $\phi_0$ and the sign $\pm$ is the sign of the charge $q$.

For small $\omega t$ we get
\begin{align*}
z_2&\approx \frac{p_\parallel}{p_\perp}\frac{1}{\sqrt{1-\frac{1}{12}\omega^2t^2}}\\
&\approx\frac{p_\parallel}{p_\perp}\left(1+\frac{1}{24}\omega^2t^2\right).
\end{align*}
Hence up to first order in $\omega t$ the coordinate $z_2$ is constant for each helix and given by
\begin{equation*}
z_2\approx \frac{p_\parallel}{p_\perp}.
\end{equation*}

We set
\begin{align*}
x_2 &= \frac{x}{\sqrt{x^2+y^2+z^2}}\\
y_2 &= \frac{y}{\sqrt{x^2+y^2+z^2}}.
\end{align*}
Then the expression
\begin{align*}
x_2^2+y_2^2 &= \frac{x^2+y^2}{x^2+y^2+z^2}\\
&=\frac{1}{1+z_2^2}
\end{align*}
is independent of $R$ and $\phi_0$.

For small $\omega t$, where $z_2\approx \frac{p_\parallel}{p_\perp}$, the point $(x_2,y_2)$ lies approximately on a circle of radius
\begin{equation*}
\frac{1}{\sqrt{1+\frac{p_\parallel^2}{p_\perp^2}}} = \frac{p_\perp}{\sqrt{p_\parallel^2+p_\perp^2}} = \frac{p_\perp}{p},
\end{equation*}
where $p$ is the total momentum.

Using Taylor expansion for small $\omega t$ we can calculate
\begin{align*}
x(t)&\approx R\sin(\phi_0)\omega t\\
y(t)&\approx -R\cos(\phi_0)\omega t.
\end{align*}
This implies
\begin{align*}
\lim_{t\rightarrow 0}x_2(t)&=\sin(\phi_0)\frac{p_\perp}{p}\\
\lim_{t\rightarrow 0}y_2(t)&=-\cos(\phi_0)\frac{p_\perp}{p}.
\end{align*}
We see that the (idealized) initial point $(x_2,y_2)$ on the circle of radius $\frac{p_\perp}{p}$ depends on the angle $\phi_0$, even though $x_0=y_0=0$. Hence in the coordinates $x_2,y_2$ the different helices get separated, depending on the angle $\phi_0$.

## The Angular Track Transformation 

The final transformation that is essential is as follows :
$$ \{x,y,z\} \rightarrow \{x_2,y_2,z_2\}`$$
Where 
$$ \{x_2,y_2,z_2\} \rightarrow \left\{ \frac{x}{\sqrt{x^2+y^2+z^2}},\frac{y}{\sqrt{x^2+y^2+z^2}},\frac{z}{\sqrt{x^2+y^2}}  \right\} $$

The slide below also mentions the $\{x,y\}$ transformation and also lists it's benifits. 
<br>
<img src ="https://i.imgur.com/SH5JXuM.png" width = 200>
<br>

## Spherical coordinates

To compare the usefulness of the above __Angulr Track Transform__,I also compared the effect of tranforming the Cartesian coordintes to the well known __Spherical Coordinate__ system. The maths behind that is as follows  

Spherical coordinates $(r,\phi,\theta)$, with $r\geq 0, \phi\in[0,2\pi], \theta\in[0,\pi]$, are given by
\begin{align*}
x &= r\sin\theta\cos\phi\\
y &= r\sin\theta\sin\phi\\
z &= r\cos\theta.
\end{align*}
Using our calculations above we can describe the helix by
\begin{align*}
r(t)&=|R|\sqrt{\left(\frac{p_\parallel}{p_\perp}\right)^2\omega^2t^2+4\sin^2\left(\frac{\omega t}{2}\right)}\\
\phi(t)&=\phi_0-\frac{1}{2}\omega t-\frac{\pi}{2}\\
\theta(t)&=\arctan\left(\frac{p_\perp}{p_\parallel}\frac{2\sin\left(\frac{\omega t}{2}\right)}{\omega t}\right).
\end{align*}
Up to terms of second order in $\omega t$ we get
\begin{align*}
r(t)&\approx \frac{p}{p_\perp}|R\omega t|\\
\phi(t)&=\phi_0-\frac{1}{2}\omega t-\frac{\pi}{2}\\
\theta(t)&\approx\arctan\left(\frac{p_\perp}{p_\parallel}\right).
\end{align*}
In particular, the angle $\theta$ is approximately constant.


## Implementing Feature Engineering and Helix Transformations

### The Angular Track Transformation
As described it is implemented in the following function. It is called dbscan_preprocess as it will later be used in the clustering step to get the scores.

In [ ]:
def dbscan_preprocess(hits):
        
        x = hits[0]
        y = hits[1]
        z = hits[2]

        r = np.sqrt(x**2 + y**2 + z**2)
        x2 = x/r
        y2 = y/r

        r = np.sqrt(x**2 + y**2)
        z2 = z/r
        
        return np.vstack((x2,y2,z2))
    

### The Spherical Coordinate Trasnform
The below function implements the described spherical transform

In [ ]:
def cart2spherical(cart):
    r = np.linalg.norm(cart, axis=0)
    theta = np.degrees(np.arccos(cart[2] / r))
    phi = np.degrees(np.arctan2(cart[1], cart[0]))
    return np.vstack((r, theta, phi))

## Plot Cartesian Coordinates Particle Trajectories

The following image is the paths of the 20 particles that have the highest truth weights, i.e. most likely to be correct. As we see many particles overlapp and some have bizzare paths. 

**Important Point:** If we notice clearly we see that not all particles originate from $\{0,0,0\}$, this is because particles are generated by the decay of other particles, hence the beginning of the particle path can be anywhere.

In [ ]:
# Get particle id with highest weights
NUM_PARTICLES = 20
truth_dedup = truth.drop_duplicates('particle_id')
truth_sort = truth_dedup.sort_values('weight', ascending=False)
truth_head = truth_sort.head(NUM_PARTICLES)

# Get points where the same particle intersected subsequent layers of the observation material
p_traj_list = []
for _, tr in truth_head.iterrows():
    p_traj = truth[truth.particle_id == tr.particle_id][['tx', 'ty', 'tz']]
    p_traj_list.append(p_traj)
    
# Convert to spherical coordinate.
rtp_list = []
ang_list = []
for p_traj in p_traj_list:
    xyz = p_traj.loc[:, ['tx', 'ty', 'tz']].values.transpose()
    rtp = cart2spherical(xyz).transpose()
    rtp_df = pd.DataFrame(rtp, columns=('r', 'theta', 'phi'))
    ang = dbscan_preprocess(xyz).transpose()
    ang_df = pd.DataFrame(ang, columns=('x2', 'y2', 'z2'))
    ang_list.append(ang_df)
    rtp_list.append(rtp_df)

# Plot with Cartesian coordinates.
fig = plt.figure(figsize=(10, 10))
ax = fig.add_subplot(111, projection='3d')
for p_traj in p_traj_list:
    ax.plot(
        xs=p_traj.tz,
        ys=p_traj.tx,
        zs=p_traj.ty,
        marker='o')
ax.set_xlabel('Z (mm) -- Detection layers')
ax.set_ylabel('X (mm)')
ax.set_zlabel('Y (mm) ')
plt.title('Trajectories of top weights particles in Cartesian coordinates.')

plt.show()



## Particle Trajectories in Spherical Coordinates $\{R,\phi,\theta\}$

We can already see that a Spherical Transformation is able to isolate out the trajectories adn this will be useful if we are to use clustering methods. But there are still erratic paths in this coordinate system

In [ ]:
# Plot with spherical coordinates.
fig2 = plt.figure(figsize=(10, 10))
ax = fig2.add_subplot(111, projection='3d')
for rtp_df in rtp_list:
    ax.plot(
        xs=rtp_df.theta,
        ys=rtp_df.phi,
        zs=rtp_df.r,
        marker='o')
ax.set_xlabel('Theta (deg)')
ax.set_ylabel('Phi (deg)')
ax.set_zlabel('R  (mm) -- Detection layers')
plt.title('Trajectories of top weights particles in spherical coordinates.')
plt.show()

## Particle Trajectories in Angular Track Coordinates $\{x_2.y_2,z_2\}$

In the below figure the hits have been transformed into the angular track coordinates that was discussed in the previous sections. We can see that the hits from the same path are quite tightly grouped and much better than the spherical coordinates. This is the major transformation that will be used in the sample submission that will be developed.

In [ ]:
# Plot with angular track coordinates.
fig3 = plt.figure(figsize=(10, 10))
ax = fig3.add_subplot(111, projection='3d')
for ang_df in ang_list:
    ax.plot(
        xs=ang_df.z2,
        ys=ang_df.x2,
        zs=ang_df.y2,
        marker='o')
ax.set_xlabel('z2')
ax.set_ylabel('x2')
ax.set_zlabel('y2')
plt.title('Trajectories of top weights particles in anglular track coordinates.')
plt.show()

## Other Feature Engineering Options
There are plenty of other feature engineering methods and approaches that are there and I didn't try out. Mostly because I wanted to understand what I was doind and not just take stuff from kernels and run for better results. Hence even though this approach only yeilds $\approx$ __0.2__ on the public leaderboard, I will leave the more advanced methods to when I have time to comprehend the Phsycis and Maths behind them. A non-exhaustive list of the other options are :
- Hough Transformations
- Kalman Filters 
- Scaled Angular Track coordinates

Now lets move onto the Algorithm and the implementation

## The Chosen Approach

As mentioned before this challenge doens't rely on advanced ML algorithms. Given good and reasonable features the approach is typically to use some variant of Clustering to group hits into particles and predict the tracks. The most common kNN clustering is also valid but doens't yield the best results. Typically __DBSCAN__ is the preferred approach for this problem.

### DBSCAN 
**DBSCAN** (Density-Based Spatial Clustering and Application with Noise), is a density-based clusering algorithm, introduced in Ester et al. 1996, which can be used to identify clusters of any shape in a data set containing noise and outliers.

Two important parameters are required for DBSCAN: 

*   epsilon (**eps**) : defines the radius of neighborhood around a point x. It’s called called the ϵ-neighborhood of x. 
*   minimum points (**MinPts**) : the minimum number of neighbors within “eps” radius.

By default the public DBSCAN benchmark used __eps=0.008__ and achieves an average score of __0.207__. A major task that can be done using ML is using the existing data to make a good guess on the value of __eps__. This improves the clustering and the overall score.


Below is an implementation of a clustering __Class__ that will preprocess the data according to the Angular Track transformation and then proceed to cluster using __DBSCAN__

In [ ]:
from sklearn.preprocessing import StandardScaler
from sklearn.cluster import DBSCAN

class Clusterer(object):
    
    def __init__(self, eps):
        self.eps = eps
        
    
    def _preprocess(self, hits):
        
        x = hits.x.values
        y = hits.y.values
        z = hits.z.values

        r = np.sqrt(x**2 + y**2 + z**2)
        hits['x2'] = x/r
        hits['y2'] = y/r

        r = np.sqrt(x**2 + y**2)
        hits['z2'] = z/r

        ss = StandardScaler()
        X = ss.fit_transform(hits[['x2', 'y2', 'z2']].values)
        
        return X
    
    
    def predict(self, hits):
        
        X = self._preprocess(hits)
        
        cl = DBSCAN(eps=self.eps, min_samples=1, algorithm='kd_tree')
        labels = cl.fit_predict(X)
        
        return labels


### Sample clustering of hits of one event 

In [ ]:
model = Clusterer(eps=0.00715)
labels= model.predict(hits)

In [ ]:
# The cluster labels generated 
print(labels)

### Function to create a submission 

In [ ]:
def create_one_event_submission(event_id, hits, labels):
    sub_data = np.column_stack(([event_id]*len(hits), hits.hit_id.values, labels))
    submission = pd.DataFrame(data=sub_data, columns=["event_id", "hit_id", "track_id"]).astype(int)
    return submission

### Checking the score of the sample clustering above

In [ ]:
submission = create_one_event_submission(0, hits, labels)
score = score_event(truth, submission)
print("The score is : ", score)

### Cross Validating 
Checking the mean score for 5 events that are occur 1000 events after the sample 1st event.
We see that we achieve an average score of $\approx$ __0.2__

In [ ]:
dataset_submissions = []
dataset_scores = []

for event_id, hits, cells, particles, truth in load_dataset(path_to_train, skip=1000, nevents=5):
        
    # Track pattern recognition
    model = Clusterer(eps=0.00715)
    labels = model.predict(hits)
        
    # Prepare submission for an event
    one_submission = create_one_event_submission(event_id, hits, labels)
    dataset_submissions.append(one_submission)
    
    # Score for the event
    score = score_event(truth, one_submission)
    dataset_scores.append(score)
    
    print("Score for event %d: %.3f" % (event_id, score))
    
print('Mean score: %.3f' % (np.mean(dataset_scores)))

## Create a Submission File
Using the test data this time we create a submission file

In [ ]:
test_dataset_submissions = []

create_submission = False # True for submission 

if create_submission:
    for event_id, hits, cells in load_dataset(path_to_test, parts=['hits', 'cells']):

        # Track pattern recognition
        model = Clusterer(eps=0.00715)
        labels = model.predict(hits)

        # Prepare submission for an event
        one_submission = create_one_event_submission(event_id, hits, labels)
        test_dataset_submissions.append(one_submission)
        
        print('Event ID: ', event_id)

    # Create submission file
    submussion = pd.concat(test_dataset_submissions, axis=0)
    submussion.to_csv('submission.csv.gz', index=False, compression='gzip')

# Results
As expected the submission scored __0.20643__ on the public leaderboard and  __0.20817__ on the private leaderboard. This is good as we haven't overfir to the public data and still perform equally well on the whole data set. 

Unfortunately this is the extent of my Kaggle competition and exploration. There are still many ML methods that I haven't ventured into, because the nature of the competiton makes it either redundant or very computationlly intensive. The are and not limited to:
- Trying XGBoost on multiple coordinate transformations as features along with particle data
- Using exploratory kNN and other approaches to optimize the __eps__ value for DBSCAN
- Utilizing track voting and ensemble-esqe methods to improve final track prediction

# Conclusions

The Kaggle Course was immensely helpful in providing tips and tricks to gain the best model given the limitations/constraints of a Kaggle Competition. The weekly topics were from a wide variety of domains and topics and with expert talks helping me understand the best practices, whether it was cross validation, model selection or ensemble methods.

Although in my specific Kaggle Challenge I wasn't able to implement many of the topics that were discussed. I really loved the __TrackML__ competition, it gave a perspective of domain specific problems that cannot be tackled by simple application of well known ML algorithms. It helped me better understand the value of a good data exploration and visualization using seaborn. These are the most valuable learning that I take away from the course and the competition in general.

__Thank You for reading this long and technical notebook__

<br>
<img src="https://i.imgur.com/8Tm5jRx.png">
<br>